In [2]:
import tensorflow as tf
import numpy as np

train_data = tf.keras.utils.image_dataset_from_directory(
    "./train",
    label_mode="categorical",
    image_size=(150, 150),
    shuffle=False,
    seed=0
)

valid_data = tf.keras.utils.image_dataset_from_directory(
    "./valid",
    label_mode="categorical",
    image_size=(150, 150),
    shuffle=False,
    seed=0
)

test_data = tf.keras.utils.image_dataset_from_directory(
    "./test",
    label_mode="categorical",
    image_size=(150, 150),
    shuffle=False,
    seed=0
)


X_train = np.concatenate([x for x, _ in train_data], axis=0)
y_train = np.concatenate([y for _, y in train_data], axis=0)

X_valid = np.concatenate([x for x, _ in valid_data], axis=0)
y_valid = np.concatenate([y for _, y in valid_data], axis=0)

X_test = np.concatenate([x for x, _ in test_data], axis=0)
y_test = np.concatenate([y for _, y in test_data], axis=0)



Found 6552 files belonging to 102 classes.
Found 818 files belonging to 102 classes.
Found 819 files belonging to 102 classes.


In [3]:
classes = train_data.class_names
n_classes = len(classes)
n_classes

102

In [4]:
# n_batches = valid_data.cardinality()
# n_batches

In [5]:
for transform in [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)]:
    
    train_data = train_data.map(lambda x, y: (transform(x), y))

In [6]:
!pip install keras-tuner

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from kerastuner.tuners import RandomSearch


def create_model(hp=None):
    base_model = Xception(
        weights="imagenet",
        input_shape=(150, 150, 3),
        include_top=False,
    )

    base_model.trainable = False

    inputs = tf.keras.Input(shape=(150, 150, 3))
    scaling = layers.Rescaling(scale=1 / 127.5, offset=-1)
    outputs = scaling(inputs)
    outputs = base_model(outputs, training=False)
    outputs = layers.Dense(256, activation='relu')(outputs)
    outputs = layers.Dense(128, activation='relu')(outputs)
    outputs = layers.GlobalAveragePooling2D()(outputs)
    outputs = layers.Dense(n_classes, activation='softmax')(outputs)
    model = Model(inputs, outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0003),
        loss=CategoricalCrossentropy(),
        metrics=[CategoricalAccuracy()]
    )

    return model

model = create_model()
model.summary(show_trainable=True)

C:\Users\Mary Mare\AppData\Local\Temp\ipykernel_8920\1871328632.py:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         Y          
                                                                            
 rescaling (Rescaling)       (None, 150, 150, 3)       0         Y          
                                                                            
 xception (Functional)       (None, 5, 5, 2048)        20861480  N          
                                                                            
 dense (Dense)               (None, 5, 5, 256)         524544    Y          
                                                                            
 dense_1 (Dense)             (None, 5, 5, 128)         32896     Y          
                                                                            
 global_average_pooling2d (G  (None, 128)              0     

In [8]:
for transform in [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]:
    train_data = train_data.map(lambda x, y: (transform(x), y))

In [9]:
# !pip install keras-tuner
import keras_tuner as kt
import numpy as np

tuner = kt.Hyperband(
    hypermodel=create_model,
    objective="categorical_accuracy",
    max_epochs=10,
    factor=3,
    project_name="flower_hp"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

tuner.search(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid), callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Reloading Tuner from .\flower_hp\tuner0.json


In [10]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=10)

205/205 [==============================] - 347s 2s/step - loss: 3.0516 - categorical_accuracy: 0.3442 - val_loss: 1.9833 - val_categorical_accuracy: 0.5232


In [11]:
val_acc_per_epoch = history.history["val_categorical_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch))

Best epoch: 1


In [12]:
hypermodel = tuner.hypermodel.build(best_hps)
history_hypermodel=hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_data=(X_valid, y_valid))

205/205 [==============================] - 363s 2s/step - loss: 3.0936 - categorical_accuracy: 0.3350 - val_loss: 2.0794 - val_categorical_accuracy: 0.5110


In [15]:
import json
import numpy as np
import tensorflow as tf

# Load the class name dictionary from the JSON file
with open('Flower-Classes-to-Flower-Names-Dictionary.json', 'r') as f:
    classes = json.load(f)

# Assuming you have defined and loaded your functional model
model = hypermodel

def predict(path):
    image = tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)

    # Use predict to get the raw prediction output
    raw_predictions = model.predict(image)

    # Get the top three predicted classes and their probabilities
    top_three_indices = np.argsort(raw_predictions[0])[::-1][:3]
    top_three_predictions = [(classes[str(index)], raw_predictions[0][index]) for index in top_three_indices]

    return top_three_predictions

# Example usage
top_three_predictions = predict("canterbury-bells-image.jpg")
print("Top three predictions:")
for i, (flower_name, probability) in enumerate(top_three_predictions, start=1):
    print(f"{i}. {flower_name}: Probability {probability}")


1/1 [==============================] - 1s 1s/step
Top three predictions:
1. mallow: Probability 0.21693263947963715
2. ruby-lipped cattleya: Probability 0.09137009084224701
3. blanket flower: Probability 0.0481443926692009


In [ ]:
# Assuming hypermodel is a Keras Functional model

# Training score
training_results = hypermodel.evaluate(X_train, y_train)
training_loss = training_results[0]  # Assuming loss is the first element in the results
training_accuracy = training_results[1]  # Assuming accuracy is the second element in the results
print(f'Training Loss: {training_loss}, Training Accuracy: {training_accuracy}')

# Test score
test_results = hypermodel.evaluate(X_test, y_test)
test_loss = test_results[0]  # Assuming loss is the first element in the results
test_accuracy = test_results[1]  # Assuming accuracy is the second element in the results
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


205/205 [==============================] - 549s 3s/step - loss: 0.1556 - categorical_accuracy: 0.9699
Training Loss: 0.15557324886322021, Training Accuracy: 0.9699328541755676
26/26 [==============================] - 69s 3s/step - loss: 0.9841 - categorical_accuracy: 0.7436
Test Loss: 0.9840571284294128, Test Accuracy: 0.7435897588729858


In [ ]:
model.save("trained-model-flower.keras")

In [ ]:
!pip install gradio

In [ ]:
%%writefile app.py
# !pip install gradio ipywidgets
import joblib
import tensorflow as tf
import gradio as gr
import numpy as np

model = tf.keras.models.load_model("model.keras")
classes = joblib.load("classes.joblib")

def predict(path):
    image = tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)

    predicted = model.predict(image)[0].argmax(axis=-1)
    return classes[predicted]

# https://www.gradio.app/guides
with gr.Blocks() as blocks:
    path = gr.Image(label="Image", type="filepath")
    label = gr.Textbox(label="Label")

    inputs = [path]
    outputs = [label]

    predict_btn = gr.Button("Predict")
    predict_btn.click(predict, inputs=inputs, outputs=outputs)

if __name__ == "__main__":
    blocks.launch() # Local machine only
    # blocks.launch(server_name="0.0.0.0") # LAN access to local machine
    # blocks.launch(share=True) # Public access to local machine
    # predict("cats_vs_dogs/cat/0.jpg")

Writing app.py


In [ ]:
%run app.py

OSError: No file or directory found at model.keras